In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import copy

## Парсинг amc

In [2]:
def read_frame(frame, frames_dict):
    # parse joints
    for joint in frame:
        joint_info = joint.strip().split()
        joint_name = joint_info[0]
        joint_info.remove(joint_name)
        
        for i in range(len(joint_info)):
            coord = joint_info[i]
            frames_dict[joint_name + '_' + str(i)] = float(coord)
            
    return frames_dict

In [3]:
def parse_amc(filepath):
    with open(filepath) as f:
        content = f.read().splitlines()

    for idx, line in enumerate(content):
        if line == ':DEGREES':
            content = content[idx+1:]
            break
            
    # get idx of frames beginning
    frame_beginning = []
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if len(splitted_line) == 1:
            frame_beginning.append(idx)
    # parse frames
    all_frames = []
    for i in range(len(frame_beginning)-1):
        frame = content[frame_beginning[i]:frame_beginning[i+1]]
        frames_dict = {}
        frames_dict['frame_number'] = frame[0]
        # parse joints
        frames_dict = read_frame(frame[1:], frames_dict)
        all_frames.append(frames_dict)

    all_data = pd.DataFrame(all_frames)            
        
    return all_data

In [4]:
content = parse_amc('../data/02_01.amc')

In [5]:
content.head()

,frame_number,head_0,head_1,head_2,lclavicle_0,lclavicle_1,lfemur_0,lfemur_1,lfemur_2,lfingers_0,...,rwrist_0,thorax_0,thorax_1,thorax_2,upperback_0,upperback_1,upperback_2,upperneck_0,upperneck_1,upperneck_2
0,1,2.97226,2.54741,-1.92752,-1.620100e-14,-1.431250e-14,-28.0191,-1.020780,-20.1783,7.12502,...,-16.7798,-0.599593,1.31944,-2.05527,3.47255,2.55226,1.65684,1.617840,5.61246,-6.40733
1,2,3.01316,2.59206,-1.92353,-3.061280e-14,-7.951390e-16,-27.7269,-0.953135,-20.5822,7.12502,...,-16.5037,0.078456,1.35992,-1.86912,3.91254,2.58553,1.67171,1.419540,5.71695,-6.43796
2,3,2.90215,2.58200,-1.86960,3.975690e-15,-1.033680e-14,-27.6281,-0.810148,-21.0041,7.12502,...,-16.2262,0.211031,1.41449,-1.83734,3.96845,2.68906,1.61541,1.081860,5.66983,-6.25219
3,4,2.73437,2.55894,-1.86943,1.376580e-14,0.000000e+00,-27.6637,-0.391122,-21.6105,7.12502,...,-16.0827,0.127573,1.45150,-1.90802,3.81888,2.77415,1.44970,0.706102,5.58519,-6.09066
4,5,2.59153,2.57436,-1.87971,-2.042510e-14,-7.156250e-15,-27.6786,-0.161279,-21.8039,7.12502,...,-16.0025,0.158931,1.46518,-1.93228,3.70601,2.80336,1.40701,0.388960,5.58739,-5.98980


In [6]:
content.shape

(342, 63)

Хотим получить словарь из данного датафрейма для удобной работы

In [55]:
cols = content.columns.tolist()
cols.remove('frame_number')

bone_type = [col.split("_")[0] for col in cols]
bone_type_set = set(bone_type)

bone_orient = {}
for bone in bone_type_set:
    bone_col_names = [col for col in cols if bone in col]
    bone_orient[bone] = bone_col_names

In [80]:
bone_orient

{'rthumb': ['rthumb_0', 'rthumb_1'],
 'rfoot': ['rfoot_0', 'rfoot_1'],
 'root': ['root_0', 'root_1', 'root_2', 'root_3', 'root_4', 'root_5'],
 'ltoes': ['ltoes_0'],
 'lclavicle': ['lclavicle_0', 'lclavicle_1'],
 'lwrist': ['lwrist_0'],
 'lthumb': ['lthumb_0', 'lthumb_1'],
 'rtoes': ['rtoes_0'],
 'rclavicle': ['rclavicle_0', 'rclavicle_1'],
 'lradius': ['lradius_0'],
 'lfoot': ['lfoot_0', 'lfoot_1'],
 'rwrist': ['rwrist_0'],
 'rhand': ['rhand_0', 'rhand_1'],
 'ltibia': ['ltibia_0'],
 'lowerneck': ['lowerneck_0', 'lowerneck_1', 'lowerneck_2'],
 'thorax': ['thorax_0', 'thorax_1', 'thorax_2'],
 'upperneck': ['upperneck_0', 'upperneck_1', 'upperneck_2'],
 'rtibia': ['rtibia_0'],
 'lhumerus': ['lhumerus_0', 'lhumerus_1', 'lhumerus_2'],
 'rradius': ['rradius_0'],
 'head': ['head_0', 'head_1', 'head_2'],
 'lfingers': ['lfingers_0'],
 'lfemur': ['lfemur_0', 'lfemur_1', 'lfemur_2'],
 'rfemur': ['rfemur_0', 'rfemur_1', 'rfemur_2'],
 'lhand': ['lhand_0', 'lhand_1'],
 'upperback': ['upperback_0', '

In [94]:
full_amc = {}
for i in range(content.shape[0]):
    fn = content.loc[i].frame_number
    amc_data = {}
    amc_frame = content[content['frame_number'] == fn]
    
    for bone, cols in bone_orient.items():
        amc_data[bone] = amc_frame[cols].values[0]
    full_amc[int(fn)] = amc_data

In [101]:
full_amc[1]['root']

array([ 10.4194 ,  16.7048 , -30.1003 ,  -2.48972,  -9.82194,  -3.00914])

## Парсинг asf

Считываем данные: иерархию и параметры сегментов

In [7]:
def is_float(s):
    try:
        float(s)
        return float(s)
    
    except ValueError:
        return False
    
def is_number(s):
    
    if s.isdigit() == False:
        float_s = is_float(s)
        if float_s == False:
            val = False
        else:
            val = float_s
    else:
        val = int(s)
    return val

def read_bone_params(bone_data):
    
    clear_bone_data = [line for line in bone_data if ('end' in line)|('id' in line)|('begin' in line) == False] 
    
    # merge in one line all "limits"
    for id, param in enumerate(clear_bone_data):
        if param.strip().split()[0] == 'limits':
            clear_bone_data_new = clear_bone_data[:id]
            new_limits = "".join(clear_bone_data[id:])
            clear_bone_data_new.append(new_limits)
            break
        else:
            clear_bone_data_new = clear_bone_data  
    
    params_dict = {}
    for line in clear_bone_data_new:
        str_data = line.strip().split()
        if str_data[0] == 'name':
            bone_name = str_data[1]   
        else:
            values = list(str_data[1:])
            values_new = []
            for s in values:
                par = is_number(s)                             
                if  type(par) != bool:
                    val = par
                else:
                    val = s
                values_new.append(val)    
            params_dict[str_data[0]] = values_new
            
    bone_params = {}
    bone_params[bone_name] = params_dict   
    
    return bone_params


def parse_asf(filepath):

    with open(filepath) as f:
        content = f.read().splitlines()

    
    for idx, line in enumerate(content):
        splitted_line = line.strip().split()
        if line.rfind('hierarchy') > -1:
            hierarchy_start=idx
        if line.rfind('bonedata') > -1:
            bonedata_start=idx
            
    # считываем нижний блок с расписанной архитектурой   
    hierarchy_content = content[hierarchy_start:]
    clear_hierarchy = []
    for idx, line in enumerate(hierarchy_content):
        if (('hierarchy' in line) | ('begin' in line) | ('end' in line)) == False:
            clear_hierarchy.append(line.strip())

    hierarchy_dict = {}        
    for line in clear_hierarchy:
        splitted_line = line.split()
        for i in range(len(splitted_line)-1):
            hierarchy_dict[splitted_line[i+1]] = splitted_line[0]
        
        
    # надо теперь посчитать сколько bones
    bone_types = []
    bone_types_id = []
    for idx, line in enumerate(content):
        if (line.rfind('name') > -1 and idx > bonedata_start):
            bone_types_id.append(idx)
            # надо теперь посчитать сколько bones
            splitted_line = line.strip().split()
            #но и сохраняем
            bone_types.append(splitted_line[1])
            
    bone_types_iter = copy.deepcopy(bone_types_id)
    bone_types_iter.append(hierarchy_start)
    all_params = {}

    for i in range(len(bone_types_iter)-1):
        bone_data = content[bone_types_iter[i]:bone_types_iter[i+1]]
        bone_params = read_bone_params(bone_data)
        all_params = dict(all_params, **bone_params)
        
    return all_params, hierarchy_dict

In [8]:
filepath = '../data/02.asf'
all_p, hierarchy_dict = parse_asf(filepath)

## Отрисовка скелета из asf

In [171]:
def length_to_vector(current_bone, all_p):
    axis = all_p[current_bone]['direction'][:3]
    child_length = all_p[current_bone]['length']
    child_vect=[[0], [0], [0]]
    for idx, axes in enumerate(axis):
        p = axes
        if p != 0:
            child_vect[idx] = child_length[0]*p
    vect_all = []
    for vect in child_vect:
        if isinstance(vect, list):
            new_vect = vect[0]
        else:
            new_vect = vect
        vect_all.append(new_vect)
    return np.array(vect_all)

trans_dict = {} #translated coordinates for bones
trans_dict['root'] = np.array([0, 0, 0]).reshape(3, 1)

def get_transform(all_p, trans_dict, child_name='lowerback', parent_name='root'):

    par_coor = trans_dict[parent_name] # coordinates of parent
    child_vect = length_to_vector(child_name, all_p) # coordinates of child
    child_vect_in_root = par_coor + child_vect.reshape(3, 1)
    return child_vect_in_root

def get_child_transform(current_bone, hierarchy_dict, all_p, trans_dict):
    childs = []
    for key, value in hierarchy_dict.items():
        if value == current_bone:
            childs.append(key)
    for child in childs:
        child_coord = get_transform(all_p, trans_dict, child_name=child, parent_name=current_bone)
        trans_dict[child] = child_coord
    return trans_dict
# for root lambda function
trans_dict = get_child_transform(current_bone='root', hierarchy_dict=hierarchy_dict, all_p=all_p, trans_dict=trans_dict)

for current, parent in hierarchy_dict.items():
    transformed_bones = list(trans_dict.keys())
    if (current not in transformed_bones) and (parent in transformed_bones):  
        trans_dict = get_child_transform(current_bone=parent, hierarchy_dict=hierarchy_dict, 
                                             all_p=all_p, trans_dict=trans_dict)

In [149]:
import plotly.plotly as py
import plotly.graph_objs as go

In [174]:
bone_list = []
values = []
value_array = np.zeros(shape=(len(trans_dict), 3))

for idx, (key, value) in enumerate(trans_dict.items()):
    bone_list.append(key)
    value_array[idx] = value.reshape(len(value))
    
# plot
trace1 = go.Scatter3d(
    x=value_array[:, 0],
    y=value_array[:, 1],
    z=value_array[:, 2],
    mode='markers+text',
    text=bone_list
)
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=10, range = [-20,20],),
                    yaxis = dict(
                        nticks=10, range = [-20,20],),
                    zaxis = dict(
                        nticks=10, range = [-20,20],),)
#                     width=700,
#                     margin=dict(
#                     r=20, l=10,
#                     b=10, t=10)
                  )
py.iplot(go.Figure(data=[trace1], layout=layout), filename='3d-scatter-colorscale')

## Отрисуем фрейм из amc
Для этого нужен как amc файл, так и asf

#### Хотим найти матрицу поворота осей и обратную к ней (считаем из asf) 

Будем использовать для нахождения координат сегментов в глобальой системе координат

In [119]:
def axis_to_matrix(current_bone, all_p):
    
    a = all_p[current_bone]['axis'][0] * np.pi / 180 #угол 20 град.
    b = all_p[current_bone]['axis'][1] * np.pi / 180 #угол 20 град.
    c = all_p[current_bone]['axis'][2] * np.pi / 180 #угол 20 град.

    Ax = np.array([[1,0,0], [0,np.cos(a),-np.sin(a)], [0,np.sin(a),np.cos(a)]])
    Ay = np.array([[np.cos(b),0,np.sin(b)], [0,1,0], [-np.sin(b),0,np.cos(b)]])
    Az = np.array([[np.cos(c),-np.sin(c),0], [np.sin(c),np.cos(c),0], [0,0,1]])
    c_matrix = np.dot(np.dot(Ax, Ay), Az) # rotation matrix around all axis
    
    return c_matrix
# axis_to_matrix('lfoot', all_p)

In [120]:
c_matrix_all = {}
c_matrix_all_inv = {}

for bone in all_p.keys():
    if bone != 'root':
        c_matrix = axis_to_matrix(bone, all_p)
        c_matrix_all[bone] = c_matrix
        c_matrix_all_inv[bone] = np.linalg.inv(c_matrix)

In [123]:
c_matrix_all

{'lhipjoint': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'lfemur': array([[ 0.93969262, -0.34202014,  0.        ],
        [ 0.34202014,  0.93969262,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 'ltibia': array([[ 0.93969262, -0.34202014,  0.        ],
        [ 0.34202014,  0.93969262,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 'lfoot': array([[ 9.39692621e-01, -3.42020143e-01,  1.33142791e-17],
        [ 8.43136386e-18,  6.20933297e-17,  1.00000000e+00],
        [-3.42020143e-01, -9.39692621e-01,  6.12323400e-17]]),
 'ltoes': array([[ 9.39692621e-01, -3.42020143e-01,  1.33142791e-17],
        [ 8.43136386e-18,  6.20933297e-17,  1.00000000e+00],
        [-3.42020143e-01, -9.39692621e-01,  6.12323400e-17]]),
 'rhipjoint': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'rfemur': array([[ 0.93969262,  0.34202014,  0.        ],
        [-0.34202014,  0.93969262,  0.        ],
        [ 0.        ,  0

for normolize amc make preparetion (add degrees of freedom and orientations)

In [104]:
trans_dict = {}  #current position for each frame (translated coordinates)
# trans_dict['root'] = np.array([0, 0, 0]).reshape(3, 1)
orient_dict = {} #current orientation for each frame
# orient_dict['root'] = np.identity(n=3)

def get_bone_data_from_amc_root(amc_data, trans_dict, orient_dict):
    root_coor = amc_data['root'][:3]
    root_coor = root_coor.reshape(3,1)
    
    root_orient = amc_data['root'][3:]
    a = root_orient[0] * np.pi / 180 #угол 20 град.
    b = root_orient[1] * np.pi / 180 #угол 20 град.
    c = root_orient[2] * np.pi / 180 #угол 20 град.

    Ax = np.array([[1,0,0], [0,np.cos(a),-np.sin(a)], [0,np.sin(a),np.cos(a)]])
    Ay = np.array([[np.cos(b),0,np.sin(b)], [0,1,0], [-np.sin(b),0,np.cos(b)]])
    Az = np.array([[np.cos(c),-np.sin(c),0], [np.sin(c),np.cos(c),0], [0,0,1]])
    orient_matrix = np.dot(np.dot(Ax, Ay), Az) # rotation matrix around all axis
    trans_dict['root'] = root_coor
    orient_dict['root'] = orient_matrix
    return trans_dict, orient_dict

def get_bone_data_from_amc_nonroot(current_bone, parent_bone, hierarchy_dict, all_p, amc_data, trans_dict, orient_dict):
    # calculate the position of child using orientation of parent and coordinates of parent
    parent_coor = trans_dict[parent_bone] # coordinates of parent
    parent_orient = orient_dict[parent_bone] # orientation matrix
    c_matrix = axis_to_matrix(parent_bone, all_p)
    c_matrix_inv = np.linalg.inv(c_matrix)
    
    child_vect = length_to_vector(child_name, all_p) # coordinates of child in local without orientation
    child_vect_oriented = length_to_vector(child_name, all_p) # + orientation of parent frame
    child_vect_in_root = par_coor + child_vect.reshape(3, 1)
    return child_vect_in_root
    # calculation of orientation for this joint
    
    current_bone = 'root'
    dofs = all_p[current_bone]['dof']
    dof_vals = full_amc[1][current_bone]
    rotation = {}
    for idx, dof in enumerate(dofs):
        rotation[dof] = 0
        if 'rx' in rotation.keys():
            a = rotation['rx'] * np.pi / 180 #угол 20 град.
        if 'ry' in rotation.keys():
            b = rotation['ry'] * np.pi / 180 #угол 20 град.
        if 'rz' in rotation.keys():
            c = rotation['rz'] * np.pi / 180 #угол 20 град.
    Ax = np.array([[1,0,0], [0,np.cos(a),-np.sin(a)], [0,np.sin(a),np.cos(a)]])
    Ay = np.array([[np.cos(b),0,np.sin(b)], [0,1,0], [-np.sin(b),0,np.cos(b)]])
    Az = np.array([[np.cos(c),-np.sin(c),0], [np.sin(c),np.cos(c),0], [0,0,1]])
    orient_matrix = np.dot(np.dot(Ax, Ay), Az) # rotation matrix around all axis
    return coor, orient_matrix

def get_transform_amc(all_p, trans_dict, child_name='lowerback', parent_name='root'):
    
    # get parent data
    par_coor, orient_matrix = get_bone_data_from_amc()
    # print("par_coor: {}".format(par_coor))

    # get child data
    child_vect = length_to_vector(child_name, all_p) # coordinates of child
#     child_axis = axis_to_matrix(child_name, all_p)
#     print(child_axis)
#     child_vect = np.dot(child_vect, child_axis) # take into accaunt the axis - uncomment it
    child_vect_in_root = par_coor + child_vect
    
    print("child_vect_in_root: {}".format(child_vect_in_root)) 
    return child_vect_in_root

def get_child_transform_amc(current_bone, hierarchy_dict, all_p, trans_dict):
    
    childs = []
    for key, value in hierarchy_dict.items():
        if value == current_bone:
            childs.append(key)

    for child in childs:
        child_coord = get_transform_amc_nonroot(all_p, trans_dict, child_name=child, parent_name=current_bone)
        trans_dict[child] = child_coord
        
    return trans_dict
## FOR ONE FRAME only!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# for root
trans_dict, orient_dict = get_bone_data_from_amc_root(amc_data=full_amc[1], trans_dict=trans_dict, orient_dict=orient_dict)

for current, parent in hierarchy_dict.items():
    print(current)
    transformed_bones = list(trans_dict.keys())
    if (current not in transformed_bones) and (parent in transformed_bones):  
        trans_dict, orient_dict = get_bone_data_from_amc_nonroot(current_bone=current, parent_bone=parent,
                                                                 hierarchy_dict=hierarchy_dict, 
                                             all_p=all_p, amc_data=full_amc[1], trans_dict=trans_dict, orient_dict=orient_dict)

In [110]:
hierarchy_dict

{'lhipjoint': 'root',
 'rhipjoint': 'root',
 'lowerback': 'root',
 'lfemur': 'lhipjoint',
 'ltibia': 'lfemur',
 'lfoot': 'ltibia',
 'ltoes': 'lfoot',
 'rfemur': 'rhipjoint',
 'rtibia': 'rfemur',
 'rfoot': 'rtibia',
 'rtoes': 'rfoot',
 'upperback': 'lowerback',
 'thorax': 'upperback',
 'lowerneck': 'thorax',
 'lclavicle': 'thorax',
 'rclavicle': 'thorax',
 'upperneck': 'lowerneck',
 'head': 'upperneck',
 'lhumerus': 'lclavicle',
 'lradius': 'lhumerus',
 'lwrist': 'lradius',
 'lhand': 'lwrist',
 'lthumb': 'lwrist',
 'lfingers': 'lhand',
 'rhumerus': 'rclavicle',
 'rradius': 'rhumerus',
 'rwrist': 'rradius',
 'rhand': 'rwrist',
 'rthumb': 'rwrist',
 'rfingers': 'rhand'}

In [107]:
trans_dict, orient_dict = get_bone_data_from_amc_root(amc_data=full_amc[1], trans_dict=trans_dict, orient_dict=orient_dict)

In [109]:
orient_dict

{'root': array([[ 0.98398403,  0.05172582, -0.17058682],
        [-0.04504561,  0.99806751,  0.04280342],
        [ 0.17247121, -0.03443369,  0.98441252]])}

#### Вспомогательные, пока не использующиеся функции

In [13]:
def make_rotation(current_bone, all_p):
    rotation_values = [20, 0, 0];# их потом будем получать из amc
    rotation = {}
    dofs = all_p[current_bone]['dof']
    for idx, dof in enumerate(dofs):
        rotation[dof] = rotation_values[idx]
    if 'rx' in rotation.keys():
        a = rotation['rx'] * np.pi / 180 #угол 20 град.
    else:
        a = 0
    if 'ry' in rotation.keys():
        b = rotation['ry'] * np.pi / 180 #угол 20 град.
    else:
        b = 0
    if 'rz' in rotation.keys():
        c = rotation['rz'] * np.pi / 180 #угол 20 град.
    else:
        c = 0

    Ax = np.array([[1,0,0], [0,np.cos(a),-np.sin(a)], [0,np.sin(a),np.cos(a)]])
    Ay = np.array([[np.cos(b),0,np.sin(b)], [0,1,0], [-np.sin(b),0,np.cos(b)]])
    Az = np.array([[np.cos(c),-np.sin(c),0], [np.sin(c),np.cos(c),0], [0,0,1]])
    child_S = np.dot(np.dot(Ax, Ay), Az) # rotation matrix around all axis
#     child_vect_in_par = np.dot(child_S, child_vect) # vector in parent frame

    return child_S
def get_dofs(current_bone, all_p):
    rotation_values = [20, 0, 0];# их потом будем получать из amc
    dofs = all_p[current_bone]['dof']
    for idx, dof in enumerate(dofs):
        rotation[dof] = rotation_values[idx]
    return idx